In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.svm import SVR
import threading
import queue
import torch.nn.functional as F
import csv
import cv2


In [ ]:

plot_queue = queue.Queue()

# 定義複雜的多項式函數及其非線性部分
def true_function(x):
    return np.where(x < -5,
              np.sin(x) + np.cos(2*x) + 0.5*x**2 - 2,
              np.where(x < 0,
                np.sin(x) + np.cos(2*x) + 0.5*x**2 - 2*x,
                np.sin(x) + np.cos(2*x) + np.log(x + 2) + np.sqrt(x) + 10))


# 生成訓練數據
x_train = np.linspace(-10, 10, 1000)
y_train = true_function(x_train)

# 生成驗證數據
x_val = np.linspace(-11, 11, 200)
y_val = true_function(x_val)

# 轉換成 PyTorch 張量
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).view(-1, 1)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32).view(-1, 1)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
x_val_tensor = x_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)


In [ ]:
# 繪製結果
plt.plot(x_train, y_train, label='True Function')
plt.legend()
plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Comparison of Regression Techniques')
plt.grid(True)

# 設定 y 軸的最大值為數值中的最大值*1.1
plt.ylim([min(y_train), max(y_train) * 1.1])

plt.show()

In [ ]:
class PolynomialApproximatorTransformer(nn.Module):
    def __init__(self, input_dim=1, d_model=128, num_heads=8, num_layers=3, dropout=0.1):
        super(PolynomialApproximatorTransformer, self).__init__()
        self.input_proj = nn.Linear(input_dim, d_model) 
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(d_model, 1)  

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.input_proj(x) 
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x) 
        x = x.permute(1, 0, 2)  
        x = self.fc_out(x[:, -1, :])  
        return x

# 創建模型
model = PolynomialApproximatorTransformer()
dummy_input = torch.randn(32, 1)  # batch_size=32, input_dim=1
output = model(dummy_input)
print(output.shape) 

model.to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [ ]:

# 創建儲存圖片的資料夾
if not os.path.exists('images'):
    os.makedirs('images')

# 定義儲存圖片的函數
def save_plot():
  while True:
    epoch, x_train, y_train, y_pred_tensor, y_pred_svr = plot_queue.get()
    if epoch is None:
        break
    plt.figure(dpi=225)
    plt.ylim([min(y_train), max(y_train) * 1.1])
    plt.plot(x_train, y_train, label='True Function')
    plt.plot(x_train, y_pred_tensor, label='Neural Network Approximation', linestyle='--')
    plt.plot(x_train, y_pred_svr, label='SVR', linestyle='--')

    plt.legend()
    plt.xlabel('x')
    plt.ylabel('f(x)')
    plt.title(f'Epoch {epoch+1}')
    plt.grid(True)

    plt.savefig(f'images/epoch_{epoch+1}.png')
    plt.close()
    plot_queue.task_done()


In [ ]:
# 訓練 SVR 模型
svr_model = SVR(kernel='rbf', C=1000, gamma=0.1)
svr_model.fit(x_train.reshape(-1, 1), y_train.ravel())
y_pred_svr = svr_model.predict(x_train.reshape(-1, 1))

# 訓練模型
num_epochs = 10000
csv_file = 'training_results.csv'
data_buffer = []

    

for epoch in range(num_epochs):
    model.train()  # 設置模型為訓練模式
    # 前向傳播
    outputs = model(x_train_tensor)
    loss = criterion(outputs, y_train_tensor)

    # 反向傳播與優化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # 驗證
    model.eval()  # 設置模型為評估模式
    with torch.no_grad():
        val_outputs = model(x_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)

        if (epoch+1) % 5 == 0:

            print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {loss.item():.4f}, Validation Loss: {val_loss.item():.4f}')

            # 預測模型的輸出（使用 eval 模式，不需要 train）
            y_pred_tensor = model(x_train_tensor).detach().cpu().numpy()

            # 將數據存入 buffer 而不是直接寫入 CSV
            for i in range(len(x_train)):
                data_buffer.append([epoch+1, x_train[i], y_train[i], y_pred_tensor[i][0], y_pred_svr[i]])

# 訓練結束後再一次性寫入 CSV
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['epoch', 'x_train', 'y_train', 'y_pred_tensor', 'y_pred_svr'])
    writer.writerows(data_buffer)

In [ ]:
from tqdm.notebook import tqdm
import os

csv_file = 'training_results.csv'


# 創建儲存圖片的資料夾
if not os.path.exists('images'):
    os.makedirs('images')



def plot_from_csv(csv_file):
    # 讀取數據時明確指定數據格式，以確保讀取為二維陣列
    data = np.genfromtxt(csv_file, delimiter=',', skip_header=1, dtype=float)

    # 檢查數據的形狀，避免索引錯誤
    if data.ndim == 1:
        data = np.expand_dims(data, axis=0)

    # 提取不同的 epoch
    epochs = np.unique(data[:, 0])

    for epoch in tqdm(epochs):
          epoch_data = data[data[:, 0] == epoch]
          x_train = epoch_data[:, 1]
          y_train = epoch_data[:, 2]
          y_pred_tensor = epoch_data[:, 3]
          y_pred_svr = epoch_data[:, 4]

          # 生成圖表
          # plt.figure(dpi=225)
          plt.ylim([min(y_train), max(y_train) * 1.1])
          plt.plot(x_train, y_train, label='True Function')
          plt.plot(x_train, y_pred_tensor, label='Neural Network Approximation', linestyle='--')
          plt.plot(x_train, y_pred_svr, label='SVR', linestyle='--')

          plt.legend()
          plt.xlabel('x')
          plt.ylabel('f(x)')
          plt.title(f'Epoch {int(epoch)}')
          plt.grid(True)

          plt.savefig(f'images/epoch_{int(epoch)}.png')
          plt.close()
          plt.clf()



# 訓練完成後，從 CSV 生成圖表
plot_from_csv(csv_file)

In [ ]:
num_epochs = 10000

frame_rate = 60 
frame_size = (640, 480) 
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

out = cv2.VideoWriter('polynomial_approximation.mp4', fourcc, frame_rate, frame_size)
for epoch in range(5, num_epochs+1, 5):
    filename = f'images/epoch_{epoch}.png'
    img = cv2.imread(filename)
    img_resized = cv2.resize(img, frame_size)
    out.write(img_resized)

out.release()
